In [1]:
!pip install datasets
!pip install indic-nlp-library
import torch
import math
import pickle
import time
import torchtext
from torch import Tensor
from datasets import load_dataset
import io
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
import torch.nn as nn
from torch.utils.data import DataLoader
import spacy
from spacy.lang.hi.examples import sentences
from torch.nn.utils.rnn import pad_sequence
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from torch.nn import (TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer)
import pandas as pd

In [2]:
# !unzip Hindi_English_Truncated_Corpus.csv
df = pd.read_csv('Hindi_English_Truncated_Corpus.csv', delimiter = ',')
source = df['english_sentence'].tolist()
target = df['hindi_sentence'].tolist()

In [3]:
source_train_file = source[0:80]
target_train_file = target[0:80]
source_valid_file = source[7001:7005]
target_valid_file = target[7001:7005]
source_test_file = source[3101:3102]
target_test_file = target[3101:3102]

In [4]:
source_train_file

['politicians do not have permission to do what needs to be done.',
 "I'd like to tell you about one such child,",
 'This percentage is even greater than the percentage in India.',
 "what we really mean is that they're bad at not paying attention.",
 '.The ending portion of these Vedas is called Upanishad.',
 'The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .',
 'In this lies the circumstances of people before you.',
 'And who are we to say, even, that they are wrong',
 '“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.',
 "You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .",
 'Please ensure that you use the appropriate form .',
 'Category: Religious Text',
 'This period summarily is pepped up with devotion.',
 'So there 

In [5]:
target_train_file

['राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .',
 'मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,',
 'यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।',
 'हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते',
 'इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।',
 'कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी .',
 'इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।',
 'और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं',
 'ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।',
 'हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . .',
 'कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प्रयोग कर रहें हैं .',
 'श्रेणी:धर्मग्रन्थ',
 'यह काल समग्रतः भक्ति भावना से ओतप्रोत काल है।',
 'तो 

In [6]:
"""factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer('hi',remove_nuktas = True)
counterx = Counter()
for x in target_train_file:
    #print(x)
    words = normalizer.normalize(x)
    #tokens = indic_tokenize.trivial_tokenize(words)
    counterx.update(words)
print(counterx)
#print(vocab(counterx))
"""

"factory=IndicNormalizerFactory()\nnormalizer=factory.get_normalizer('hi',remove_nuktas = True)\ncounterx = Counter()\nfor x in target_train_file:\n    #print(x)\n    words = normalizer.normalize(x)\n    #tokens = indic_tokenize.trivial_tokenize(words)\n    counterx.update(words)\nprint(counterx)\n#print(vocab(counterx))\n"

In [7]:
en_tokenizer = get_tokenizer('spacy', language = 'en_core_web_sm')
def hi_tokenizer(data):
    factory=IndicNormalizerFactory()
    normalizer=factory.get_normalizer('hi',remove_nuktas = True)
    text = normalizer.normalize(data)
    words = indic_tokenize.trivial_tokenize(text)
    return words

In [8]:
def build_vocab(filedata, tokenizer):
    counter = Counter()
    for string_ in filedata:
        #print(string_)
        counter.update(tokenizer(string_))
    print(counter)
    return(vocab(counter, specials = ['<unk>', '<pad>', '<bos>', '<eos>']))

In [9]:
en_vocab = build_vocab(source_train_file,en_tokenizer)
en_vocab.set_default_index(en_vocab['<unk>'])
en_vocab[' ']

Counter({'the': 74, '.': 66, ',': 55, 'of': 45, 'to': 44, 'and': 35, 'is': 24, 'in': 17, 'that': 17, 'with': 16, 'a': 16, '-': 13, 'on': 11, 'this': 10, 'not': 9, 'you': 9, 'for': 9, 'be': 8, 'are': 8, "'s": 8, 'or': 7, 'can': 7, 'his': 7, 'This': 6, 'British': 6, 'has': 6, 'as': 6, 'what': 5, 'one': 5, 'child': 5, 'they': 5, 'The': 5, 'was': 5, 'In': 5, 'by': 5, 'it': 5, 'he': 5, 'India': 4, 'we': 4, 'but': 4, 'its': 4, 'your': 4, 'up': 4, 'were': 4, 'their': 4, 'from': 4, 'time': 4, 'will': 4, 'her': 4, 'do': 3, 'I': 3, 'even': 3, 'than': 3, 'mean': 3, 'at': 3, 'people': 3, 'And': 3, '”': 3, 'school': 3, ':': 3, 'some': 3, 'had': 3, 'would': 3, 'years': 3, 'after': 3, 'more': 3, 'progress': 3, 'our': 3, 'which': 3, 'But': 3, 'He': 3, 'all': 3, 'female': 3, 'been': 3, 'us': 3, 'have': 2, 'needs': 2, 'such': 2, 'percentage': 2, 'these': 2, 'Governor': 2, 'before': 2, 'who': 2, '“': 2, 'refer': 2, 'caused': 2, 'being': 2, 'may': 2, 'want': 2, 'special': 2, 'an': 2, 'meet': 2, 'use': 2, 

0

In [10]:
hi_vocab = build_vocab(target_train_file,hi_tokenizer)
hi_vocab.set_default_index(hi_vocab['<unk>'])
hi_vocab['<eos>']

Counter({'के': 61, 'है': 48, '.': 46, 'की': 39, ',': 32, 'और': 32, 'में': 29, 'से': 25, 'का': 25, 'को': 22, '।': 20, 'हैं': 20, 'कि': 14, 'यह': 11, 'भी': 11, 'पर': 11, 'लिए': 11, 'इस': 10, 'या': 10, 'जो': 9, 'नहीं': 9, 'हुए': 9, 'आप': 9, 'अपने': 9, '-': 9, 'ही': 8, 'एक': 8, 'कर': 8, ')': 8, '(': 7, 'पास': 6, 'करने': 6, 'ये': 6, 'ने': 6, 'किया': 6, 'हो': 6, 'गया': 6, 'तक': 6, 'साथ': 6, 'तथा': 6, 'वह': 5, 'अधिक': 5, 'हम': 5, 'इसमें': 5, 'वाले': 5, 'किसी': 5, 'कारण': 5, 'सकते': 5, 'उनके': 5, 'करते': 5, 'ब्रिटिश': 5, 'था': 4, 'इसके': 4, 'तो': 4, 'कम': 4, 'एवं': 4, 'समय': 4, 'बाद': 4, 'बहुत': 4, 'थे': 4, 'प्रतिशत': 3, 'चाहते': 3, 'पूर्व': 3, 'लोगों': 3, 'वे': 3, 'हुई': 3, 'रहने': 3, 'सकता': 3, 'विशेष': 3, 'प्रति': 3, 'बताया': 3, 'वर्ष': 3, 'न्यायालय': 3, 'कुछ': 3, 'धर्म': 3, 'होने': 3, 'जाती': 3, 'रहे': 3, 'दिया': 3, 'हमारी': 3, 'प्रेम': 3, 'स्वतंत्रता': 3, 'महिला': 3, 'उसकी': 3, 'हमें': 3, 'इंडिया': 3, 'कार्य': 2, 'करना': 2, 'चाहिए': 2, 'अनुमति': 2, 'बारे': 2, 'भारत': 2, 'वो': 2, 'दे': 2, 

3

In [11]:
print(en_vocab['beautiful'])

0


In [12]:
def data_process(filepath_source,filepath_target):
    raw_en_iter = iter(filepath_source)
    raw_hi_iter = iter(filepath_target)
    data = []
    for (raw_en, raw_hi) in zip(raw_en_iter, raw_hi_iter):
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en.rstrip("n"))],dtype = torch.long)
        hi_tensor_ = torch.tensor([hi_vocab[token] for token in hi_tokenizer(raw_hi.rstrip("n"))],dtype = torch.long)
        data.append((en_tensor_, hi_tensor_))
    return data

In [13]:
train_data = data_process(source_train_file,target_train_file)
val_data = data_process(source_valid_file, target_valid_file)
test_data = data_process(source_test_file, target_test_file)

In [14]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

80
4
1


In [15]:
BATCH_SIZE = 128
PAD_IDX = en_vocab['<pad>']
BOS_IDX = en_vocab['<bos>']
EOS_IDX = en_vocab['<eos>']
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
def generate_batch(data_batch):
    en_batch, hi_batch = [], []
    for (en_item, hi_item) in data_batch:
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim = 0))
        hi_batch.append(torch.cat([torch.tensor([BOS_IDX]), hi_item, torch.tensor([EOS_IDX])], dim = 0))
    en_batch = pad_sequence(en_batch, padding_value = PAD_IDX)
    hi_batch = pad_sequence(hi_batch, padding_value = PAD_IDX)
    return en_batch, hi_batch

In [17]:
train_iter = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True, collate_fn = generate_batch)
valid_iter = DataLoader(val_data, batch_size = BATCH_SIZE, shuffle = True, collate_fn = generate_batch)
test_iter = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True, collate_fn = generate_batch)

In [18]:
print(len(train_iter))
for (idx, batch) in enumerate(valid_iter):
    print(idx)
    print(batch)

1
0
(tensor([[  2,   2,   2,   2],
        [ 52,  52,  52, 110],
        [  0,   0,   0, 641],
        [380,   0,  47,   9],
        [  6,   0,   0,   0],
        [  0,  47,  27,  14],
        [  0, 414,   0,   3],
        [ 58,  67,   0,   1],
        [ 32,  59, 168,   1],
        [ 31,  37,   0,   1],
        [  0,   0,   0,   1],
        [ 37, 288,   9,   1],
        [685,   0,   0,   1],
        [177,  59,   0,   1],
        [279,   0,  86,   1],
        [ 24, 573, 220,   1],
        [ 31,   9,  27,   1],
        [  0,   0,   0,   1],
        [  0,   0,   0,   1],
        [111,   0,   0,   1],
        [656, 110,   0,   1],
        [  0,   0,  31,   1],
        [ 14,   0,   0,   1],
        [  3, 415,  47,   1],
        [  1, 174,   0,   1],
        [  1,   0,  14,   1],
        [  1, 194,   3,   1],
        [  1, 205,   1,   1],
        [  1,   0,   1,   1],
        [  1,  14,   1,   1],
        [  1,   3,   1,   1]]), tensor([[  2,   2,   2,   2],
        [ 29,  55,   0, 114],
   

In [19]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers:int, num_decoder_layers:int, emb_size:int, src_vocab_size:int, tgt_vocab_size:int, dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model = emb_size, nhead = NHEAD, dim_feedforward = dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers = num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model = emb_size, nhead = NHEAD, dim_feedforward = dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers = num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout = dropout)

    def forward(self, src: Tensor, tgt: Tensor, src_mask:Tensor, tgt_mask:Tensor, src_padding_mask:Tensor, tgt_padding_mask:Tensor, memory_key_padding_mask:Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src:Tensor, src_mask:Tensor):
        return self.transformer_encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt:Tensor, memory:Tensor, tgt_mask:Tensor):
        return self.transformer_decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [20]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size:int, dropout, maxlen:int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0,emb_size, 2) * math.log(10000)/emb_size)
        pos = torch.arange(0,maxlen).reshape(maxlen,1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:,0::2] = torch.sin(pos*den)
        pos_embedding[:,1::2] = torch.cos(pos*den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding',pos_embedding)

    def forward(self, token_embedding:Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0),:])

In [21]:
class TokenEmbedding(nn.Module):
    def __init__(self,vocab_size:int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens:Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [22]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz,sz), device = DEVICE)) == 1).transpose(0,1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [23]:
def create_mask(src,tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device = DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0,1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0,1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [24]:
SRC_VOCAB_SIZE = len(en_vocab)
TGT_VOCAB_SIZE = len(hi_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 32
BATCH_SIZE = 1
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
NUM_EPOCHS = 150
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [25]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim()>1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index = PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr = 0.0001, betas = (0.9,0.98), eps = 1e-9)

In [26]:
def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src,tgt) in enumerate(train_iter):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1,:]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1,logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    torch.save(model,'EN-HI Translator.pth')
    return losses/len(train_iter)

In [27]:
def evaluate(model,val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1,:]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses/ len(val_iter)

In [28]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch(transformer, train_iter, optimizer)
    end_time = time.time()
    val_loss = evaluate(transformer, valid_iter)
    print((f"Epoch : {epoch}, Train loss: {train_loss:.3f}, Val Loss: {val_loss:.3f},"
           f"Epoch Time= {(end_time - start_time):.3f}s"))


C:\Users\arshk\anaconda3\Lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch : 1, Train loss: 7.088, Val Loss: 6.612,Epoch Time= 28.890s
Epoch : 2, Train loss: 6.471, Val Loss: 6.783,Epoch Time= 20.234s
Epoch : 3, Train loss: 6.233, Val Loss: 6.933,Epoch Time= 22.334s
Epoch : 4, Train loss: 6.095, Val Loss: 7.019,Epoch Time= 20.545s
Epoch : 5, Train loss: 6.043, Val Loss: 7.090,Epoch Time= 22.984s
Epoch : 6, Train loss: 5.985, Val Loss: 7.117,Epoch Time= 21.583s
Epoch : 7, Train loss: 5.905, Val Loss: 7.110,Epoch Time= 22.858s
Epoch : 8, Train loss: 5.884, Val Loss: 7.066,Epoch Time= 20.564s
Epoch : 9, Train loss: 5.831, Val Loss: 7.005,Epoch Time= 21.383s
Epoch : 10, Train loss: 5.801, Val Loss: 6.925,Epoch Time= 20.930s
Epoch : 11, Train loss: 5.764, Val Loss: 6.841,Epoch Time= 20.227s
Epoch : 12, Train loss: 5.723, Val Loss: 6.784,Epoch Time= 22.309s
Epoch : 13, Train loss: 5.660, Val Loss: 6.768,Epoch Time= 20.707s
Epoch : 14, Train loss: 5.624, Val Loss: 6.788,Epoch Time= 19.123s
Epoch : 15, Train loss: 5.570, Val Loss: 6.835,Epoch Time= 19.387s
Epoc

Epoch : 124, Train loss: 1.011, Val Loss: 6.464,Epoch Time= 17.710s
Epoch : 125, Train loss: 0.991, Val Loss: 6.464,Epoch Time= 17.708s
Epoch : 126, Train loss: 0.965, Val Loss: 6.533,Epoch Time= 17.604s
Epoch : 127, Train loss: 0.964, Val Loss: 6.513,Epoch Time= 17.790s
Epoch : 128, Train loss: 0.938, Val Loss: 6.424,Epoch Time= 17.575s
Epoch : 129, Train loss: 0.888, Val Loss: 6.403,Epoch Time= 17.707s
Epoch : 130, Train loss: 0.884, Val Loss: 6.458,Epoch Time= 17.556s
Epoch : 131, Train loss: 0.879, Val Loss: 6.457,Epoch Time= 19.071s
Epoch : 132, Train loss: 0.855, Val Loss: 6.418,Epoch Time= 21.552s
Epoch : 133, Train loss: 0.855, Val Loss: 6.406,Epoch Time= 24.383s
Epoch : 134, Train loss: 0.818, Val Loss: 6.410,Epoch Time= 21.810s
Epoch : 135, Train loss: 0.791, Val Loss: 6.413,Epoch Time= 22.655s
Epoch : 136, Train loss: 0.781, Val Loss: 6.418,Epoch Time= 18.029s
Epoch : 137, Train loss: 0.748, Val Loss: 6.420,Epoch Time= 15.318s
Epoch : 138, Train loss: 0.730, Val Loss: 6.417,

In [29]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1,1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len -1):
        memory = memory.to(DEVICE)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(DEVICE).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0,1)
        prob = model.generator(out[:,-1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1,1).type_as(src.data).fill_(next_word)],dim = 0)
        if next_word == EOS_IDX:
            break
    return ys

In [30]:
tok = 'a'
something = en_vocab[tok]
something

101

In [31]:
def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    itos = tgt_vocab.get_itos()
    tokens = [BOS_IDX] + [src_vocab[tok] for tok in src_tokenizer(src)] + [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens,1))
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len = num_tokens + 5, start_symbol = BOS_IDX).flatten()
    return " ".join([itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [32]:
text = "slow"
output = translate(transformer, text, en_vocab, hi_vocab, en_tokenizer)
print(f'English: {text} \nHindi: {output}')

English: slow 
Hindi:  मेन 


In [34]:
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio
tts = gTTS(text)
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)

In [37]:
# STORE THE MODEL AS A PICKLE OBJECT
# pickle_out= open("model_trained2.p","wb")  # wb = WRITE BYTE
# pickle.dump(transformer,pickle_out)
# pickle_out.close()
# # cv2.waitKey(0)
# import pickle
# from your_nlp_library import YourNLPModel  # Replace with the actual NLP model import

# # Assuming you have trained or loaded your NLP model
# nlp_model = YourNLPModel()

# # Pickle the model
with open("model_trained22.pkl", "wb") as pickle_out:
    pickle.dump(transformer, pickle_out)

print("Model pickled and saved successfully.")


Model pickled and saved successfully.
